# Merge Bloomberg and Yahoo Finance Data

In [1]:
import pandas as pd

In [2]:
# Set up paths
input_dir = 'inputs/'
output_dir = 'outputs/'

# Data files
merged_final_analyzing_path = input_dir + 'Merged_Analyzing_Data.csv'

In [3]:
bloombergESG = pd.read_csv('inputs/Bloomberg_Joined_Final.csv')
yfinanceESG = pd.read_csv('inputs/Yahoo_Finance_ESG_Final.csv')

In [4]:
bloombergESG.rename(columns={'Dates': 'Date'}, inplace=True)
bloombergESG['Date'] = pd.to_datetime(bloombergESG['Date'])
yfinanceESG['Date'] = pd.to_datetime(yfinanceESG['Date'])
bloombergESG['Ticker'] = bloombergESG['Ticker'].str.split(' ').str[0]

In [5]:
yfinanceESG = yfinanceESG.sort_values(by=['Ticker', 'Date'], ascending=[True, True])

In [6]:
# Replace NaN values in ESG score columns with previous non-null value for the same company
yfinanceESG[['yf_ESG_Score', 'yf_E_Score', 'yf_S_Score', 'yf_G_Score']] = yfinanceESG.groupby('Ticker')[['yf_ESG_Score', 'yf_E_Score', 'yf_S_Score', 'yf_G_Score']].fillna(method='ffill')

In [7]:
yfinanceESG.drop('Name', axis=1, inplace=True)

In [8]:
analyzingdf = pd.merge(bloombergESG,yfinanceESG,on=['Ticker','Date'],   how='left',  
                validate='many_to_many',
                indicator=True
                )

In [9]:
yfinanceESG= yfinanceESG.drop(index=range(42297))

In [10]:
yfinanceESG['Date'] = yfinanceESG['Date'] + pd.offsets.MonthEnd(-1)  # subtract 1 month and get end of month

In [11]:
analyzingdf = pd.merge(bloombergESG,yfinanceESG,on=['Ticker','Date'],   how='left',  
                validate='many_to_many',
                indicator=True
                )
analyzingdf['ret'] = analyzingdf.groupby('Ticker')['Last Price'].pct_change()

In [12]:
analyzingdf.to_csv(merged_final_analyzing_path, index=False)